# Notebook de preparação do dataset de Acidentes

### Imports

In [ ]:
import pandas as pd
from resolve_path import ajuste_path

### Leitura do dataset de entrada pre-processado

In [ ]:
path = "data/util/"
path = ajuste_path(path)

df_acidentes_padronizado = pd.read_csv(
    path + "acidentes/" + "acidentes_preprocessado.csv", encoding="utf-8", sep="#"
)

df_acidentes_padronizado.head(10)

In [ ]:
# Criando uma nova coluna "Indice acidente" com o valor do índice de cada linha
df_acidentes_padronizado['indice acidente'] = [
    i for i in df_acidentes_padronizado.index]

### Ordenando por tempo

In [ ]:
# Ordenando o dataframe pelas datas, do mais antigo para o mais recente
df_acidentes_padronizado = df_acidentes_padronizado.sort_values(by='data')

# Reset do index
df_acidentes_padronizado = df_acidentes_padronizado.reset_index(drop=True)

### Criando o indice dos acidentes originais antes de retirar algumas categorias

### Removendo acidentes de terceiro

In [ ]:
print("Linhas antes da remoção de terceiros: ",
      df_acidentes_padronizado.shape[0])
qtd_linhas_com_terceiros = df_acidentes_padronizado[
    df_acidentes_padronizado["empregado"] == "terceiro"
].shape[0]

print("Removendo " + str(qtd_linhas_com_terceiros) + " linhas com terceiros")

df_acidentes_padronizado = df_acidentes_padronizado[
    df_acidentes_padronizado['empregado'] == 'proprio']

print("Linhas após a remoção de terceiros: ",
      df_acidentes_padronizado.shape[0])

### Removendo linhas cuja Classificação é de trajeto

In [ ]:
# contando quantas linhas possuem "trajeto" na coluna Classificação
conta_linhas_trajeto = (
    df_acidentes_padronizado["classificacao"].str.contains("trajeto").sum()
)

print(
    f"Quantidade de linhas que possuem 'trajeto' na coluna Classificação: {conta_linhas_trajeto} de {df_acidentes_padronizado.shape[0]} linhas totais."
)

# removendo as linhas que possuem "trajeto" na coluna Classificação
df_acidentes_padronizado = df_acidentes_padronizado[
    ~df_acidentes_padronizado["classificacao"].str.contains("trajeto")
]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

### Removendo acidentes que ocorreram em alguma empresa que não é a Eletrosul

In [ ]:
df_acidentes_padronizado = df_acidentes_padronizado[
    df_acidentes_padronizado["empresa"] == "cgt eletrosul"
]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

## Removendo acidentes sem ID

In [ ]:
qtd_nans_id = df_acidentes_padronizado["id"].isna().sum()

print("Quantidade de NaNs na coluna ID:", qtd_nans_id)

df_acidentes_padronizado = df_acidentes_padronizado[
    (~df_acidentes_padronizado["id"].isna()) & (
        df_acidentes_padronizado["id"] != '-')
]

## Eliminação das linhas de acidentes irrelevantes e de predisposição física

In [ ]:
print(
    f"Quantidade de linhas antes a remoção: {df_acidentes_padronizado.shape[0]}")

acidentes_irrelevantes = [
    {"id": 1540378, "data": "2021-01-08 09:00:00"},
    {"id": 1542316, "data": "2021-08-10 07:00:00"},
    {"id": 1539115, "data": "2022-04-20 10:00:00"},
    {"id": 1532135, "data": "2022-08-19 14:00:00"},
    {"id": 1533092, "data": "2023-07-16 12:00:00"},
    {"id": 1541615, "data": "2024-02-07 09:00:00"},
    {"id": 210344, "data": "2024-03-01 08:00:00"},
    {"id": 9002643, "data": "2024-03-22 14:00:00"},
    {"id": 9002552, "data": "2023-05-29 11:00:00"},
    {"id": 1523324, "data": "2023-07-25 10:00:00"},
]

id_acidentes_irrelevantes = [acidente["id"]
                             for acidente in acidentes_irrelevantes]
data_acidentes_irrelevantes = [acidente["data"]
                               for acidente in acidentes_irrelevantes]

df_acidentes_padronizado["id"] = df_acidentes_padronizado["id"].astype(int)

df_acidentes_padronizado = df_acidentes_padronizado.loc[~
                                                        (df_acidentes_padronizado["id"].isin(id_acidentes_irrelevantes)
                                                         & df_acidentes_padronizado["data"].isin(data_acidentes_irrelevantes))
                                                        ]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

### Selecionando quais colunas ficarão

In [ ]:
# columns = ['id', 'data', 'descricao', 'classificacao', 'latitude', 'longitude', 'agente causador']
# df_acidentes_padronizado = df_acidentes_padronizado[columns]

### Salvando o novo dataset

In [ ]:
# Salva o dataset no folder modificados
path = "data/util/"
path = ajuste_path(path)

df_acidentes_padronizado.to_csv(
    path + "acidentes/" + "acidentes_preparado.csv",
    sep="#",
    encoding="utf-8",
    index=False,
)